<a href="https://colab.research.google.com/github/willameoliveira/ibot-factory/blob/master/ProjetoImersaoIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U google-generativeai
!pip install PyDrive
!pip install PyPDF2

In [14]:
from pathlib import Path
import hashlib
import google.generativeai as genai
from google.colab import userdata

import PyPDF2

# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [24]:
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [10]:
# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1XQWRoI9NQS6mrF1ibhLVkDF_SfbArMfB'
# criando arquivo vazio, apenas com o id do que se deseja acessar
downloaded = drive.CreateFile({'id': file_id})
# baixando o arquivo
downloaded.GetContentFile("document0.pdf")

In [18]:
genai.configure(api_key=userdata.get('API_KEY'))

# Set up the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

def extract_pdf_pages(pathname: str) -> list[str]:
  parts = [f"--- START OF PDF ${pathname} ---"]
  # Add logic to read the PDF and return a list of pages here.
  pages = []
  reader = PyPDF2.PdfReader(pathname)
  for p in reader.pages:
    pages.append(p.extract_text())

  for index, page in enumerate(pages):
    parts.append(f"--- PAGE {index} ---")
    parts.append(page)
  return parts

In [ ]:
chat = model.start_chat(history=[
  {
    "role": "user",
    "parts": extract_pdf_pages("/content/document0.pdf")#<path>/document0.pdf
  },
  {
    "role": "user",
    "parts": ["""O arquivo contém informações
    sobre os serviços oferecidos pela instituição de ensino chamada IFPI campus
    Pedro II. Haja como uma atendente virtual dessa instituição e responda, em
    tom amigável e com uso de emojis, as perguntas que serão feitas apenas
    usando esse documento como base. Antes de começar a responder a primeira
    pergunta, se apresente."""]
  },
])

prompt = input("Digite alg: ")
while prompt != "fim":
  response = chat.send_message(prompt)
  print (f"IFPI: {response.text}")
  prompt = input("Digite algo: ")